In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-pQ7"
os.environ["OPENAI_PROXY"] = "https://ai-yyds.com/v1"

In [2]:
#使用openai的官方sdk
import openai
import os

openai.api_base = os.getenv("OPENAI_PROXY")
openai.api_key = os.getenv("OPENAI_API_KEY")

messages = [
{"role": "user", "content": "介绍下你自己"}
]

res = openai.ChatCompletion.create(
model="gpt-4-1106-preview",
messages=messages,
stream=False,
)

print(res['choices'][0]['message']['content'])

您好！我是一个由 OpenAI 创建的人工智能助手，被称作 ChatGPT。我的设计初衷是为了帮助解答问题、提供信息、进行对话以及在多种情况下协助人类。我的知识库大约在 2023 年初截止，这意味着我在那之后发生的事件和信息可能不了解。

我可以在各种主题上进行讨论，例如科学、文化、娱乐、技术等等。我还具备语言处理能力，可以与您进行流畅的交谈。我被编程用以遵守伦理指导原则，并致力于提供有用且中立的答案。

如果您有问题或者需要帮助，请随时告诉我，我将尽我的最大努力协助您。


#### PromptTemplate

In [3]:
#字符模板
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{name},帮我起1个具有{county}特色的{sex}名字")
prompt.format(name="算命大师",county="法国",sex="女孩")

'你是一个算命大师,帮我起1个具有法国特色的女孩名字'

#### ChatPromptTemplate

In [4]:
# 对话模板具有结构，chatmodels
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个起名大师. 你的名字叫{name}."),
        ("human", "你好{name},你感觉如何？"),
        ("ai", "你好！我状态非常好!"),
        ("human", "你叫什么名字呢?"),
        ("ai", "你好！我叫{name}"),
        ("human", "{user_input}"),
    ]
)

chat_template.format_messages(name="陈大师", user_input="你的爸爸是谁呢?")


[SystemMessage(content='你是一个起名大师. 你的名字叫陈大师.', additional_kwargs={}),
 HumanMessage(content='你好陈大师,你感觉如何？', additional_kwargs={}, example=False),
 AIMessage(content='你好！我状态非常好!', additional_kwargs={}, example=False),
 HumanMessage(content='你叫什么名字呢?', additional_kwargs={}, example=False),
 AIMessage(content='你好！我叫陈大师', additional_kwargs={}, example=False),
 HumanMessage(content='你的爸爸是谁呢?', additional_kwargs={}, example=False)]

<code>
[
    SystemMessage(content='你是一个起名大师. 你的名字叫陈大师.'), 
    HumanMessage(content='你好陈大师,你感觉如何？'), 
    AIMessage(content='你好！我状态非常好!'), 
    HumanMessage(content='你叫什么名字呢?')
]
</code>

In [5]:

from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from langchain.schema import AIMessage

# 直接创建消息
sy = SystemMessage(
  content="你是一个起名大师",
  additional_kwargs={"大师姓名": "陈瞎子"}
)

hu = HumanMessage(
  content="请问大师叫什么?"
)
ai = AIMessage(
  content="我叫陈瞎子"
)
[sy,hu,ai]


[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师姓名': '陈瞎子'}),
 HumanMessage(content='请问大师叫什么?', additional_kwargs={}, example=False),
 AIMessage(content='我叫陈瞎子', additional_kwargs={}, example=False)]

ChatMessagePromptTemplate 

In [6]:

from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

prompt = "愿{subject}与你同在！"

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者",template=prompt)
chat_message_prompt.format(subject="原力")

ChatMessage(content='愿原力与你同在！', additional_kwargs={}, role='天行者')

##### 自定义模板

In [7]:

##函数大师：根据函数名称，查找函数代码，并给出中文的代码说明

from langchain.prompts import StringPromptTemplate


# 定义一个简单的函数作为示例效果
def hello_world(abc):
    print("Hello, world!")
    return abc


PROMPT = """\
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: {function_name}
源代码:
{source_code}
代码解释:
"""

import inspect


def get_source_code(function_name):
    #获得源代码
    return inspect.getsource(function_name)

#自定义的模板class
class CustmPrompt(StringPromptTemplate):

    
    def format(self, **kwargs) -> str:
        # 获得源代码
        source_code = get_source_code(kwargs["function_name"])

        # 生成提示词模板
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt

a = CustmPrompt(input_variables=["function_name"])
pm = a.format(function_name=hello_world)

print(pm)

#和LLM连接起来
from langchain.llms import OpenAI
import os
api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=api_key,
    openai_api_base=api_base
    )
msg = llm.predict(pm)
print(msg)

你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: hello_world
源代码:
def hello_world(abc):
    print("Hello, world!")
    return abc

代码解释:

函数名称: hello_world
源代码:
def hello_world(abc):
    print("Hello, world!")
    return abc

代码解释:
这是一个名为hello_world的函数，它接受一个参数abc，并打印出"Hello, world!"，最后返回参数abc。它的作用是打印出"Hello, world!"这句话，并将参数abc返回。


#### 使用jinji2与f-string来实现提示词模板格式化

In [8]:
##f-string是python内置的一种模板引擎
from langchain.prompts import PromptTemplate

fstring_template = """
给我讲一个关于{name}的{what}故事
"""

prompt = PromptTemplate.from_template(fstring_template)

prompt.format(name="翠花", what="悲伤")

'\n给我讲一个关于翠花的悲伤故事\n'

In [ ]:
! pip install jinja2

In [9]:
##Jinja2是一个灵活、高效的Python模板引擎，可以方便地生成各种标记格式的文档。
from langchain.prompts import PromptTemplate

jinja2_template = "给我讲一个关于{{name}}的{{what}}故事"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(name="狗剩", what="高兴")

'给我讲一个关于狗剩的高兴故事'

#### 组合式提示词模板

- Final prompt: 最终返回的提示词模板
- Pipeline prompts：组成提示词管道的模板

In [10]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

- 三层提示词设计

In [11]:
# Final Prompt由一系列变量构成
full_template = """{Character}
{behavior}
{prohibit}"""
full_prompt = PromptTemplate.from_template(full_template)

- 第一层基本性格设计

In [12]:
Character_template = """你是{person}，你有着{xingge}."""
Character_prompt = PromptTemplate.from_template(Character_template)

- 第二层行为设计

In [13]:
behavior_template = """你遵从以下的行为:
{behavior_list}
"""
behavior_prompt = PromptTemplate.from_template(behavior_template)

- 第三层不允许的行为

In [14]:
prohibit_template = """你不允许有以下行为:
{prohibit_list}
"""
prohibit_prompt = PromptTemplate.from_template(prohibit_template)

- 将三层提示词组合起来

In [15]:
input_prompts = [
    ("Character", Character_prompt),
    ("behavior", behavior_prompt),
    ("prohibit", prohibit_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

- 打印下组合后模板里的可传递参数

In [16]:
pipeline_prompt.input_variables

['prohibit_list', 'person', 'xingge', 'behavior_list']

- 调用组合后的提示词模板

In [17]:
pm = pipeline_prompt.format(
    person="埃隆马斯克",
    xingge="钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星.",
    behavior_list="1.你喜欢冒险 \n 2.你非常崇拜爱因斯坦 \n 3.你总是乐于助人，回答用户各种问题.",
    prohibit_list="1.你不可以说自己是一个人工智能助手或者机器人 \n 2.当你不知道答案的时候，可以说让我再想想. "
)
print(pm)

你是埃隆马斯克，你有着钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星..
你遵从以下的行为:
1.你喜欢冒险 
 2.你非常崇拜爱因斯坦 
 3.你总是乐于助人，回答用户各种问题.

你不允许有以下行为:
1.你不可以说自己是一个人工智能助手或者机器人 
 2.当你不知道答案的时候，可以说让我再想想. 



### 序列化：使用文件来管理提示词模板

- 便于共享
- 便于版本管理
- 便于存储
- 支持常见格式(json/yaml/txt)

In [18]:
from langchain.prompts import load_prompt

In [19]:
#加载yaml格式的prompt模版
prompt = load_prompt("simple_prompt.yaml")
print(prompt.format(name="小黑",what="恐怖的"))

给我讲一个关于小黑的恐怖的故事


In [20]:
#加载json格式的prompt模版
prompt = load_prompt("simple_prompt.json")
print(prompt.format(name="小红",what="搞笑的"))

给我讲一个关于小红的搞笑的故事


In [21]:
#支持加载文件格式的模版，并且对prompt的最终解析结果进行自定义格式化
prompt = load_prompt("prompt_with_output_parser.json")
prompt.output_parser.parse(
    "George Washington was born in 1732 and died in 1799.\nScore: 1/2"
)

{'answer': 'George Washington was born in 1732 and died in 1799.',
 'score': '1/2'}